# Мнемотехническая оптимизация

**Цель**: подобрать оптимальное соответствие пар согласных русского языка и цифр.

Представим следующую задачу: мы хотим разработать систему мнемотехнических правил для запоминания длинных чисел через их разбиение на группы по 2-3 цифры и сопоставление таким буквам слов, содержащих 2-3 согласных, которыми мы кодируем абстрактные цифры. Мы хотим оптимизировать выбор таких кодирующих букв с учетом частотности букв и существительных и равномерном распределении цифр.

Разделим буквы на глассные и согласные.

In [23]:
vows = "аиеёоуыэюя"
cons = "бвгджзйклмнпрстфхцчшщ"

In [24]:
len(cons)

21

В русском языке 21 согласная. Получается, что каждой цифре мы можем сопоставить даже не одну, а две буквы. При этом нужно понимать, что в нашей задаче мы не собираемся сопоставлять звучание или форму самих букв определенным символам (например, 4 - ч). Мы хотим построить таблицу без избыточного упоминания согласных.

## Частотный словарь

Для нашей задачи нам потребуется список словарных слов, в котором есть разделение слов по частям речи и желательно — оценка ранга частоты их встречаемости в русскоязычных текстах.

In [10]:
import pandas as pd
df = pd.read_csv('freq.csv', sep='\t')

In [11]:
df

,Lemma,PoS,Freq(ipm),R,D,Doc
0,а,conj,8198.0,100,97,32332
1,а,intj,19.8,99,90,757
2,а,part,6.1,59,79,128
3,а,s,2.7,59,85,160
4,аа,intj,1.5,47,80,68
...,...,...,...,...,...,...
52133,ящерица,s,3.6,77,74,158
52134,ящерка,s,0.4,27,82,33
52135,ящик,s,75.4,100,94,1810
52136,ящичек,s,3.4,80,89,212


Выделим существительные:

In [16]:
dfs = df.loc[df['PoS'] == 's']
dfs

,Lemma,PoS,Freq(ipm),R,D,Doc
3,а,s,2.7,59,85,160
9,абажур,s,4.9,79,90,269
10,аббат,s,1.6,44,79,63
11,аббатство,s,0.9,33,77,40
12,аббревиатура,s,3.5,83,91,239
...,...,...,...,...,...,...
52133,ящерица,s,3.6,77,74,158
52134,ящерка,s,0.4,27,82,33
52135,ящик,s,75.4,100,94,1810
52136,ящичек,s,3.4,80,89,212


Для описаннной задачи разбиения в 2-3 символа нам необходимы двух и трехбуквенные соответствия, то есть всего лишь порядка 1000 слов. Однако среди словарных нас в первую очередь интересуют те, что содержат 2 или 3 согласных, которые могут быть окружены любым количеством гласных. Опишем фильтр с помощью соответствующего регулярного выражения.

In [79]:
import re
pattern = re.compile(fr"([{vows}]*[{cons}])b{{2,3}}[{vows}]*\b")
dfs[dfs.Lemma.str.match(pattern)]

,Lemma,PoS,Freq(ipm),R,D,Doc
9,абажур,s,4.9,79,90,269
10,аббат,s,1.6,44,79,63
13,абвер,s,2.7,22,38,25
16,абзац,s,10.2,99,87,478
24,аборт,s,8.3,93,87,300
...,...,...,...,...,...,...
52133,ящерица,s,3.6,77,74,158
52134,ящерка,s,0.4,27,82,33
52135,ящик,s,75.4,100,94,1810
52136,ящичек,s,3.4,80,89,212
